In [1]:
import glob
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import seaborn as sns

In [2]:
dir_path = r"C:/Users/KIIT/Desktop\pneumonia_detection\chest_xray"

In [3]:
''' training path '''
train_p = os.path.join(dir_path, "train")

''' test path '''
test_p =os.path.join(dir_path, "test")

''' val path '''
val_p = os.path.join(dir_path, "val")

In [4]:
''' PNEUMONIA images '''
p_train_images = glob.glob(train_p + "/PNEUMONIA/*.jpeg")

''' NORMAL  images '''
n_train_images = glob.glob(train_p + "/NORMAL/*.jpeg")

In [5]:
df = pd.DataFrame(np.concatenate([[0]*len(n_train_images) , [1] *  len(p_train_images)]),columns=["class"])
df

,class
0,0
1,0
2,0
3,0
4,0
...,...
5211,1
5212,1
5213,1
5214,1


In [6]:
#sns.countplot(df['class'],data=df)

In [7]:
''' Data Augmentation '''
train_ds = ImageDataGenerator(rescale =1/255, shear_range=10, zoom_range = 0.2, horizontal_flip = True, 
                              width_shift_range=0.2, fill_mode = 'nearest')

val_datagen = ImageDataGenerator(
    rescale = 1/255
)

In [8]:
''' VGG16 Model '''

model1 = VGG16(include_top = False,weights='imagenet')

''' freezing layers '''
for i in model1.layers:
    i.trainable=False
    
out = model1.output

''' GlobalAveragePooling Layer '''
out = keras.layers.GlobalAveragePooling2D()(out)
out = keras.layers.Dense(128, activation='relu')(out)

''' Classification Layer '''
pred = keras.layers.Dense(1, activation='sigmoid')(out)

''' final model '''
model= keras.Model(model1.input, pred)

lr = 0.1
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(lr, decay_steps=100000, decay_rate=0.96, staircase=True)

''' Compile the model '''
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.RMSprop(lr_schedule), metrics=['accuracy'])

In [9]:
''' lets see how model looks like '''
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0     

In [10]:
''' flow from directory '''
train_ds = train_ds.flow_from_directory(train_p, target_size=(150,150), batch_size= 16, class_mode='binary')

val_ds = val_datagen.flow_from_directory(val_p, target_size=(150,150),batch_size=8, class_mode='binary')

test_ds = val_datagen.flow_from_directory(test_p, target_size=(150,150), batch_size=32, class_mode='binary')

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [ ]:
''' training '''
history = model.fit(train_ds, steps_per_epoch = 100, epochs=20, validation_data=val_ds)

Epoch 1/20
100/100 [==============================] - 288s 3s/step - loss: 3.9959 - accuracy: 0.7350 - val_loss: 0.7672 - val_accuracy: 0.5000
Epoch 2/20
100/100 [==============================] - 281s 3s/step - loss: 0.3959 - accuracy: 0.8175 - val_loss: 0.3855 - val_accuracy: 0.8750
Epoch 3/20
100/100 [==============================] - 291s 3s/step - loss: 0.3737 - accuracy: 0.8450 - val_loss: 0.2610 - val_accuracy: 0.8750
Epoch 4/20
100/100 [==============================] - 301s 3s/step - loss: 0.3852 - accuracy: 0.8575 - val_loss: 0.3753 - val_accuracy: 0.8750
Epoch 5/20
100/100 [==============================] - 303s 3s/step - loss: 0.2912 - accuracy: 0.8856 - val_loss: 0.3084 - val_accuracy: 0.9375
Epoch 6/20
100/100 [==============================] - 288s 3s/step - loss: 0.2897 - accuracy: 0.8925 - val_loss: 0.3736 - val_accuracy: 0.8125
Epoch 7/20
100/100 [==============================] - 273s 3s/step - loss: 0.2726 - accuracy: 0.9013 - val_loss: 0.2422 - val_accuracy: 0.9375

In [ ]:
''' evaluating model '''
accuracy = model.evaluate_generator(test_ds,624)[1]

In [ ]:
accuracy

In [ ]:
''' unfreezing layers '''
for layer in model1.layers:
    layer.trainable = True

In [ ]:
'''Finetuning the model'''
lr = 1e-5
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(lr, decay_steps=100000, decay_rate=0.96, staircase=True)

''' compile the model '''
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr_schedule), loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])

In [ ]:
''' training '''
model.fit(train_ds, steps_per_epoch = 50, epochs=100, validation_data=val_s)

In [ ]:
''' evaluating model '''
final_acc = model.evaluate_generator(test_generator,624)[1]

In [ ]:
final_acc

In [ ]:
''' Training Accuracy and Validaion Accuracy graph '''
plt.figure(figsize=(15,10))
plt.subplot(2, 2, 1)
plt.plot(history.history['accuracy'], label = "Training accuracy")
plt.plot(history.history['val_accuracy'], label="Validation accuracy")
plt.legend()
plt.title("Training vs validation accuracy")

''' Training Loss and Validation Loss graph '''
plt.subplot(2,2,2)
plt.plot(history.history['loss'], label = "Training loss")
plt.plot(history.history['val_loss'], label="Validation loss")
plt.legend()
plt.title("Training vs validation loss")

plt.show()